<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/TFTimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pandas_datareader

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from datetime import date
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.contrib.learn import ModeKeys
import tensorflow.contrib.rnn as rnn
from datetime import datetime, date, timedelta





In [0]:

from pandas.tseries.offsets import BDay
TIMESERIES_COL = 'rawdata'
N_OUTPUTS = 5 # in each sequence, 1-14 are features, and 14-20 is label 
SEQ_LEN = 20
DEFAULTS = 0.0
LSTM_SIZE = 5 # number of hidden layers in each of the LSTM cells
N_INPUTS = SEQ_LEN - N_OUTPUTS
BATCH_SIZE = 20
ROOT_DIR = '/home/mmistroni/tf_logs/rnn-run-{}'


def get_prices(startdate, enddate, symbol):
  import pandas_datareader as pdr
  from datetime import date, timedelta
  print ('--Start:{%s}, end:{%s}', startdate, enddate)
  
  stock_data = pdr.get_data_yahoo(symbol, startdate, enddate)[['Close']].pct_change().fillna(0)
  adjClose = np.stack(stock_data['Close'])
  return adjClose
  
def create_training_data2(inputData):
  print ('Len of input dat ais {}', len(inputData))
  return [np.array(inputData[i * SEQ_LEN: (i + 1) * SEQ_LEN]) 
       for i in range(len(inputData) // SEQ_LEN)]

def create_training_data(inputData):
  print ('AdjClose is of shape {}', inputData.shape)
  return inputData.T.reshape(-1,SEQ_LEN)
  
  
def create_time_series(end_date, numDays=360, symbol='XOM'):
  print("==== CREATING TIMESERIES GOING BACK 280 DAYS FROM  {}".format(end_date))
  endDate = end_date # training from 20 days ago
  startDate = endDate - BDay(numDays)
  prices =  get_prices(startDate, endDate, symbol=symbol)
  return create_training_data2(prices)
  


In [0]:

def create_train_and_test(end_date, symbol='XOM'):
  all_timeseries =  create_time_series(end_date, symbol=symbol) #[create_time_series() for i in range(0, SEQ_LEN * 4)]
  all_data = np.stack(all_timeseries)
  print('All data shape is{0} and type {1}'.format(all_data.shape,type(all_data)))
  X, y = all_data[...,0:-N_OUTPUTS], all_data[...,-N_OUTPUTS:]
  print ('X is fo type {0}, y  of type {1}'.format(type(X[0][0]), type(y)))
  print ('X.shape is {0}, y shap is {1}'.format(X.shape, y.shape))
  X_train, X_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      test_size=0.1,
                                                      random_state=1)
  return X_train, y_train, X_test, y_test



<h3> Creating RNN Model </h3>


In [0]:

# create the inference model
def simple_rnn(features, labels, mode, params):
  # 0. Reformat input shape to become a sequence
  print ('IN Features are:{0}'.format(features))
  x = tf.split(features[TIMESERIES_COL], N_INPUTS, 1)
  #print 'x={}'.format(x)
    
  # 1. configure the RNN
  lstm_cell = rnn.BasicLSTMCell(LSTM_SIZE, forget_bias=1.0)
  outputs, _ = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32)

  # slice to keep only the last cell of the RNN
  outputs = outputs[-1]
  #print 'last outputs={}'.format(outputs)
  
  # output is result of linear activation of last layer of RNN
  weight = tf.Variable(tf.random_normal([LSTM_SIZE, N_OUTPUTS]))
  bias = tf.Variable(tf.random_normal([N_OUTPUTS]))
  predictions = tf.matmul(outputs, weight) + bias
    
  # 2. loss function, training/eval ops
  if mode == ModeKeys.TRAIN or mode == ModeKeys.EVAL:
     loss = tf.losses.mean_squared_error(labels, predictions)
     train_op = tf.contrib.layers.optimize_loss(
         loss=loss,
         global_step=tf.train.get_global_step(),
         learning_rate=0.01,
         optimizer="SGD") # SGD
     eval_metric_ops = {
      "rmse": tf.metrics.root_mean_squared_error(labels, predictions)
     }
  else:
     loss = None
     train_op = None
     eval_metric_ops = None
  
  # 3. Create predictions
  predictions_dict = {"predicted": predictions}

  # 4. Create export outputs  
  export_outputs = {"predicted": tf.estimator.export.PredictOutput(predictions)}

  # 5. return ModelFnOps
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=eval_metric_ops,
      export_outputs=export_outputs)



<h3> Creating Serving Function , Train Function and Test Function </h3>

In [0]:
def serving_input_receiver_fn():
  feature_placeholders = {
    TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
  }

  features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }

  features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2], name='timeseries')
  
  print('serving: features={}'.format(features[TIMESERIES_COL]))

  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)



# Creating a TrainFn and a TestFn
def _train_fn(X, y, batch_size):
    
    def _train():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        # TODO need to be refactored according to https://medium.com/google-cloud/how-to-do-time-series-prediction-using-rnns-and-tensorflow-and-cloud-ml-engine-2ad2eeb189e8
        # this is not good.
        X_32 = tf.cast(X, tf.float32)
        y_32 = tf.cast(y, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        label = tf.concat(y_32, axis=1)
        features, labels = {TIMESERIES_COL: inputs}, label
        
        return features, labels
        dataset = tf.data.Dataset.from_tensor_slices(features, labels)
        # Shuffle, repeat, and batch the examples.
        dataset = dataset.repeat(None).batch(batch_size)
        # This will now return batches of features, label
        return dataset.make_one_shot_iterator().get_next()
    return _train

def _test_fn(X, y, batch_size):
    def _test():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        X_32 = tf.cast(X, tf.float32)
        y_32 = tf.cast(y, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        label = tf.concat(y_32, axis=1)
        
        features, labels = {TIMESERIES_COL: inputs}, label
        
        return features, labels
        
        dataset = tf.data.Dataset.from_tensor_slices(features, labels)
        # Shuffle, repeat, and batch the examples.
        dataset = dataset.repeat(1).batch(batch_size)
        # This will now return batches of features, label
        return dataset.make_one_shot_iterator().get_next()
    return _test  
  

 


<h3> Creating Experiment Function and running model </h3>

In [0]:
def _predict_fn(X) :
    def _predict():
        """An input function for training"""
        # Convert the inputs to a Dataset.
        X_32 = tf.cast(X, tf.float32)
        inputs = tf.concat(X_32, axis=1)
        features = {TIMESERIES_COL: inputs}
        return features
    return _predict




def experiment_fn(output_dir, X, y, x_tst, y_tst):
    # run experiment
    train_spec = tf.estimator.TrainSpec(
          input_fn=_train_fn(X, y, BATCH_SIZE), max_steps=1500)
    exporter = tf.estimator.FinalExporter('timeseries',
    serving_input_receiver_fn)
    eval_spec = tf.estimator.EvalSpec(
            input_fn=_test_fn(x_tst, y_tst, BATCH_SIZE),
            exporters=[exporter])
    
    estimator = tf.estimator.Estimator(model_fn=simple_rnn, model_dir=output_dir)

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    return estimator


def run_model(end_date, symbol='XOM'):  
  print('Attempting to generate learning model for {}, using end_date={}'.format(symbol, end_date))
  tf.logging.set_verbosity(tf.logging.INFO)    
  output_dir = ROOT_DIR.format(datetime.utcnow().strftime('%Y%m%d%H%M%S'))  
  X_train, y_train, X_test, y_test = create_train_and_test(end_date, symbol=symbol)
  estimator = experiment_fn(output_dir, X_train, y_train, X_test, y_test)
  print('Xtrain is:{} and of shape:{}', type(X_train), X_train.shape)
  return estimator


<h3> SETTING STARTDATE, ENDDATE AND TICKER </h3>

In [0]:
SYMBOL =  'RUSL'#'DIS'#'XOM'
END_DATE= date.today()
START_DATE = END_DATE - BDay(N_INPUTS) # This will be used for predicting future prices. end date will be 15 days ago, start_date 280 days earlier


<h3> Now running model </h3>

In [0]:
def train_estimator(symbol):
  print('===================== CONFIGURATION FOR {}======================'.format(symbol))
  print('STARTDATE={}'.format(START_DATE))
  print('ENDDATE={}'.format(END_DATE))
  print('SYMBOL={}'.format(SYMBOL))
  print('=== TRAINING MODEL ON PAST 280 DAYS from {}'.format(START_DATE))
  estimator = run_model(START_DATE, symbol=symbol)
  return estimator


<h3> Now, after training,  Let's do some predictions for future </h3>

In [0]:
from datetime import date
from pprint import pprint
# going back to last month
tf.logging.set_verbosity(tf.logging.INFO)    

def get_last_20_days(symbol):

  def get_prices2(startdate=None, enddate=None, symbol=''):
    import pandas_datareader as pdr
    from datetime import date, timedelta
    print ('--Start:{%s}, end:{%s}', startdate, enddate)

    if not startdate and not enddate:
      last_check = date.today() - BDay(SEQ_LEN * 2) 
      enddate = last_check
      startdate = last_check - BDay(122)#timedelta(days=120)
    return pdr.get_data_yahoo(symbol, startdate, enddate)[['Close']]


  print('==== GETTING MOST RECENT PRICES======')
  stock_data = get_prices2(START_DATE- BDay(1), END_DATE, symbol=SYMBOL)
  percentage_changes = stock_data.pct_change().fillna(0)
  #First Price
  first_price = stock_data.values[0]
  print('==============First ever price of the series of shape {} is {}'.format(stock_data.shape, first_price ))
  #print(stock_data.head(30))
  #print('=========== PERCENTAGE CHANGES ===========')
  #print(percentage_changes.head(30))
  prices = np.stack(percentage_changes['Close'])
  #prices = np.stack(stock_data['Close'])
  print('===== PRICES ARE===')
  pprint(prices)
  return prices, first_price




<h3> Now recalculating price percentages to predict </h3>

In [0]:
def make_predictions(prices, estimator, ITERATIONS=3):

  print ('Prices is of shape:{}', prices.shape)
  prices = prices.reshape(-1,N_INPUTS) if prices.shape[0] % 15 == 0 else prices[-15:].reshape(-1,N_INPUTS)
  print ('Reshaped Prices is of shape:{}', prices.shape)
  print ('-------- PREDICTING -------')
  print (type(prices))
  acc = []
  acc += prices[0].tolist()
  for i in range(0,ITERATIONS):
    # Given last 15 days, predict the next 5
    candidate = np.array(acc[-15:]).reshape(-1,15)  # this is the Train #
    pred = estimator.predict(input_fn = _predict_fn(candidate))
    item = next(pred)
    vals = item['predicted'] # this is the predicted
    # Next 5 predicted. Now,  we need to retrain the model to add the 5 predicted
    # Then, as test data, pick the previous 15 and then predict the next one

    """
    ATTEMPT TO RETRAIN THE MODEL..
      print("====== Now adding prediction to current train....")
      print("Xtrain of type:{}, candidate of type:{}".format(type(X_train), type(candidate)))
      print("SHapes.Train:{}, Candidate:{}".format(X_train.shape, candidate.shape))
      #X_train = np.vstack([X_train , candidate])
      #y_train = np.vstack([y_train , vals])
      # So Now we have to 
      # 1. retrain the model
      # 2. get the most recent 15 days to 

    """
    print ('Round {} Prediction: on {}={}'.format(i, candidate, vals))
    print ('{}={}'.format(type(vals), vals.tolist()))
    acc += vals.tolist()
  return acc



<h3> Now visualizing price predictions against date time </h3>

In [26]:
def calculate_list_of_prices(idx, first_price, zipped, accumulator):
  if idx ==0:
    ts = zipped[idx][1]
    change = zipped[idx][0]
    price = first_price * (1 + change)
    accumulator.append((ts, change, price))
    return calculate_list_of_prices(idx + 1, first_price, zipped, accumulator)
  elif idx < len(zipped):
    ts = zipped[idx][1]
    _, _, last_price = accumulator[-1]
    change =zipped[idx][0]
    accumulator.append((ts, change,last_price * (1 + change))) 
    return calculate_list_of_prices(idx + 1, first_price, zipped, accumulator)
  else:
    return accumulator
  

def generate_results(acc, first_price):
  print('-------------- END OF STORY.Computing increase from {}'.format(first_price))
  from pprint import pprint
  future_days = len(acc) - 15
  pprint('Accumlator has length:{}'.format(len(acc)))
  pprint('We have prediction for the next:{} days'.format(future_days))
  print("calculating business days to zip ")
  # we got start date, todays - 15
  # end date is  today + 
  dts = [START_DATE + BDay(offset) for offset in range(0, len(acc))]
  print('Start Date:{}={}'.format(dts[0], START_DATE))
  print('End date:{}'.format(dts[-1]))
  zipped = [tpl for tpl in zip(acc, dts)]
       
  print('===== FULL PREDICTIONS======')
  #print(zipped)
  from functools import reduce
  final_predictions  = acc
  #pprint(final_predictions)
  # Then find the end price by using reduce and entering the first price 
  res = reduce(lambda acc,x: acc * (1+x) ,  final_predictions, first_price)
  print('Final Price for series{} = {}'.format(zipped[-1], res))
  
  final_prices = calculate_list_of_prices(0, first_price, zipped, [])
  print('Final list of prices...')
  #print(final_prices)
  return res, final_prices

estimator = train_estimator(SYMBOL)
latest_prices, first_price = get_last_20_days(SYMBOL)
# Reshaping data
acc = make_predictions(latest_prices, estimator)
res, final_prices = generate_results(acc, first_price)
pprint(final_prices)



===================== CONFIGURATION FOR RUSL======================
STARTDATE=2019-06-28 00:00:00
ENDDATE=2019-07-19
SYMBOL=RUSL
=== TRAINING MODEL ON PAST 280 DAYS from 2019-06-28 00:00:00
Attempting to generate learning model for RUSL, using end_date=2019-06-28 00:00:00
==== CREATING TIMESERIES GOING BACK 280 DAYS FROM  2019-06-28 00:00:00
--Start:{%s}, end:{%s} 2018-02-09 00:00:00 2019-06-28 00:00:00


I0719 21:16:09.045658 139971056322432 estimator.py:1790] Using default config.
I0719 21:16:09.047338 139971056322432 estimator.py:209] Using config: {'_model_dir': '/home/mmistroni/tf_logs/rnn-run-20190719211608', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4d3fcf1710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker

Len of input dat ais {} 348
All data shape is(17, 20) and type <class 'numpy.ndarray'>
X is fo type <class 'numpy.float64'>, y  of type <class 'numpy.ndarray'>
X.shape is (17, 15), y shap is (17, 5)
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(15, 15) dtype=float32>}


I0719 21:16:10.286213 139971056322432 estimator.py:1147] Done calling model_fn.
I0719 21:16:10.289505 139971056322432 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0719 21:16:10.573066 139971056322432 monitored_session.py:240] Graph was finalized.
I0719 21:16:10.667391 139971056322432 session_manager.py:500] Running local_init_op.
I0719 21:16:10.693603 139971056322432 session_manager.py:502] Done running local_init_op.
I0719 21:16:11.291797 139971056322432 basic_session_run_hooks.py:606] Saving checkpoints for 0 into /home/mmistroni/tf_logs/rnn-run-20190719211608/model.ckpt.
I0719 21:16:11.817113 139971056322432 basic_session_run_hooks.py:262] loss = 0.13349324, step = 1
I0719 21:16:12.322520 139971056322432 basic_session_run_hooks.py:692] global_step/sec: 197.696
I0719 21:16:12.324241 139971056322432 basic_session_run_hooks.py:260] loss = 0.0037074257, step = 101 (0.507 sec)
I0719 21:16:12.474765 139971056322432 basic_session_run_hooks.py:692] global_step/sec: 656.844
I

IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(2, 15) dtype=float32>}


...

Round 0 Prediction: on [[-0.02319259  0.03005006 -0.01116512  0.01784738 -0.01001964  0.01572382
  -0.02206409  0.0389374  -0.02171625 -0.00733978 -0.00108208 -0.01733163
  -0.00771637  0.02129238 -0.02628718]]=[-0.0064723   0.01449966  0.00482178 -0.01263653 -0.01288748]
<class 'numpy.ndarray'>=[-0.006472304463386536, 0.014499664306640625, 0.00482177734375, -0.012636534869670868, -0.01288747787475586]
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(1, 15) dtype=float32>}


I0719 21:16:18.777305 139971056322432 estimator.py:1147] Done calling model_fn.
I0719 21:16:18.851520 139971056322432 monitored_session.py:240] Graph was finalized.
I0719 21:16:18.858831 139971056322432 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190719211608/model.ckpt-1500
I0719 21:16:18.895515 139971056322432 session_manager.py:500] Running local_init_op.
I0719 21:16:18.903935 139971056322432 session_manager.py:502] Done running local_init_op.
W0719 21:16:18.979166 139971056322432 estimator.py:1000] Input graph does not use tf.data.Dataset or contain a QueueRunner. That means predict yields forever. This is probably a mistake.
I0719 21:16:18.980515 139971056322432 estimator.py:1145] Calling model_fn.


Round 1 Prediction: on [[ 0.01572382 -0.02206409  0.0389374  -0.02171625 -0.00733978 -0.00108208
  -0.01733163 -0.00771637  0.02129238 -0.02628718 -0.0064723   0.01449966
   0.00482178 -0.01263653 -0.01288748]]=[-0.00559005  0.01353967  0.00480404 -0.01116707 -0.01302947]
<class 'numpy.ndarray'>=[-0.0055900514125823975, 0.013539671897888184, 0.004804044961929321, -0.011167071759700775, -0.013029471039772034]
IN Features are:{'rawdata': <tf.Tensor 'concat:0' shape=(1, 15) dtype=float32>}


I0719 21:16:19.482737 139971056322432 estimator.py:1147] Done calling model_fn.
I0719 21:16:19.559020 139971056322432 monitored_session.py:240] Graph was finalized.
I0719 21:16:19.564591 139971056322432 saver.py:1280] Restoring parameters from /home/mmistroni/tf_logs/rnn-run-20190719211608/model.ckpt-1500
I0719 21:16:19.599308 139971056322432 session_manager.py:500] Running local_init_op.
I0719 21:16:19.609147 139971056322432 session_manager.py:502] Done running local_init_op.


Round 2 Prediction: on [[-0.00108208 -0.01733163 -0.00771637  0.02129238 -0.02628718 -0.0064723
   0.01449966  0.00482178 -0.01263653 -0.01288748 -0.00559005  0.01353967
   0.00480404 -0.01116707 -0.01302947]]=[-0.00548412  0.01346993  0.00470009 -0.01093069 -0.01305151]
<class 'numpy.ndarray'>=[-0.0054841190576553345, 0.013469934463500977, 0.00470009446144104, -0.01093069463968277, -0.013051509857177734]
-------------- END OF STORY.Computing increase from [55.18999863]
'Accumlator has length:30'
'We have prediction for the next:15 days'
calculating business days to zip 
Start Date:2019-06-28 00:00:00=2019-06-28 00:00:00
End date:2019-08-08 00:00:00
===== FULL PREDICTIONS======
Final Price for series(-0.013051509857177734, Timestamp('2019-08-08 00:00:00')) = [51.79820652]
Final list of prices...
[(Timestamp('2019-06-28 00:00:00'),
  -0.023192585815311584,
  array([53.90999985])),
 (Timestamp('2019-07-01 00:00:00'), 0.030050063744575084, array([55.52999878])),
 (Timestamp('2019-07-02 00